### Tabla de Contenidos
**Objetivos:**
- 1. Abrir y examinar el archivo de datos.
- 2. Segmentar los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.
- 3. Investigar la calidad de diferentes modelos cambiando los hiperparámetros. Describir brevemente los hallazgos del estudio.
- 4. Comprobar la calidad del modelo usando el conjunto de prueba.
- 5. Realizar prueba de cordura al modelo.
- 6. Conclusión general

## Abrir y examinar el archivo de datos

In [ ]:
# importamos las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
# Cargamos el DataFrame
df = pd.read_csv("/datasets/users_behavior.csv")

In [ ]:
# Visualizamos el DataFrame
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [ ]:
# Revisamos su información general
print(df.info())
print()
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None

(3214, 5)


En este caso nuestro *target* sería la columna **is_ultra**. Las demás filas son los *features*. Por otra parte, este DataFrame consta de 3214 muestras.

## Segmentar los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba

Primero separamos los *features* y el *target*.

In [ ]:
# target
target_col = "is_ultra"

# features matrix
feature_cols = df.drop(columns=["is_ultra"]).columns.values


Ahora dividiremos el conjunto de datos en conjunto de entrenamiento (60%), conjunto de prueba (20%) y de validación (20%).

In [ ]:
# dividimos el conjunto de entrenamiento (60%), prueba (20%) y validación (20%)

## fijamos semilla aleatoria
random_seed = 200

## segmentamos el conjunto principal y conjunto de prueba
x_train, x_test, y_train, y_test = train_test_split(df[feature_cols],
                                                    df[target_col],
                                                    test_size = 0.2,
                                                    random_state=random_seed)

## segmentamos el conjunto de entrenamiento y el conjunto de validación
x_rtrain, x_valid, y_rtrain, y_valid = train_test_split(x_train,
                                                        y_train,
                                                        test_size = 0.25,
                                                        random_state=random_seed+1)

In [ ]:
# Revisamos las proporciones
for i in [x_rtrain, x_valid, x_train, x_test]:
  print(i.shape)

(1928, 4)
(643, 4)
(2571, 4)
(643, 4)


## Investigar la calidad de diferentes modelos cambiando los hiperparámetros. Describir brevemente los hallazgos del estudio

Para este estudio examinaremos la calidad de lo modelos *Random Forest* y *Logictic Regression*

<div class="alert alert-block alert-info">
<b>Respuesta del estudiante:</b> Entendido! <a class="tocSkip"></a>
</div>

### Comenzaremos con Random Forest.

Entrenaremos el modelo con distintos hiperparámetros para saber cuál da el *accuracy score* mayor.

In [ ]:
random_seed = 200
i = -1
best_accuracy_score = 0
best_max_depth = 0
best_n_estimators = 0

for max_depth in [1, 2, 3, 5, 6, 7, 8, 9]:
  for n_estimators in [1, 2, 3, 4, 5, 10, 20, 25]:
    i += 1
    model = RandomForestClassifier(random_state=random_seed + i,
                                   n_estimators=n_estimators,
                                   max_depth=max_depth)
    model.fit(x_rtrain, y_rtrain)
    predictions = model.predict(x_valid)
    accuracy_ = accuracy_score(y_valid, predictions)


    if accuracy_ > best_accuracy_score:
      best_max_depth = max_depth
      best_n_estimators = n_estimators
      best_accuracy_score = accuracy_

      print(f"Best max depth: {best_max_depth}")
      print(f"Best n estimators: {best_n_estimators}")
      print(f"Best accuracy score: {round(best_accuracy_score, 4)}")
      print()


Best max depth: 1
Best n estimators: 1
Best accuracy score: 0.7418

Best max depth: 1
Best n estimators: 2
Best accuracy score: 0.7574

Best max depth: 1
Best n estimators: 3
Best accuracy score: 0.7605

Best max depth: 1
Best n estimators: 4
Best accuracy score: 0.7869

Best max depth: 2
Best n estimators: 2
Best accuracy score: 0.7916

Best max depth: 2
Best n estimators: 25
Best accuracy score: 0.7978

Best max depth: 5
Best n estimators: 1
Best accuracy score: 0.804

Best max depth: 5
Best n estimators: 25
Best accuracy score: 0.8056

Best max depth: 7
Best n estimators: 5
Best accuracy score: 0.8134

Best max depth: 7
Best n estimators: 10
Best accuracy score: 0.8212

Best max depth: 9
Best n estimators: 25
Best accuracy score: 0.8243



En este caso la mejor configuración es n_estimators=25, max_depth=9, con una presición de acierto de 82,43% en el conjunto de validación.

### Seguimos con LogisticRegression

In [ ]:
# Entrenamos nuestro modelo probando hiperparámetros hasta alcanzar el mayor *Accuracy_score*
random_seeds=123
lr_model = LogisticRegression(random_state=random_seeds)
lr_model.fit(x_rtrain, y_rtrain)
predictions1 = lr_model.predict(x_valid)
accuracy1 = accuracy_score(y_valid, predictions1)
print(f"Best accuracy score: {round(accuracy1, 4)}")

Best accuracy score: 0.7465


En este caso probamos varios hiperparámetros que solo bajaban la precisión del modelo, de este modo dejamos los parámetros en default (a excepción de 'random_state'), obteniendo un 74% de precisión.

## Comprobar la calidad del modelo usando el conjunto de prueba
Ahora usaremos nuestro objeto estimator para predecir sobre las observaciones que el modelo NO vio durante el entrenamiento.
### Calidad modelo RandomForestClassifier

In [ ]:
random_seed = 42

final_model = RandomForestClassifier(random_state=random_seed,
                                   n_estimators=best_n_estimators,
                                   max_depth=best_max_depth)
final_model.fit(x_train, y_train)
#test_preds = final_model.predict(x_test)
test_preds = (final_model.predict_proba(x_test)[:,1] > 0.90).astype("int")

test_accuracy_ = accuracy_score(y_test, test_preds)

print(f"Valid accuracy score: {round(best_accuracy_score, 4)}")
print()
print(f"Test accuracy score: {round(test_accuracy_, 4)}")

Valid accuracy score: 0.8243

Test accuracy score: 0.7434


Como se puede observar, el desempeño del modelo en el conjunto de prueba ha bajado a un 74%. Esto puede deberse a un sobreajuste.

### Calidad modelo LogisticRegression

In [ ]:
random_seed = 42

final_model1 = lr_model = LogisticRegression(random_state=random_seeds)

final_model1.fit(x_train, y_train)
test_preds1 = final_model1.predict(x_test)
#test_preds1 = (final_model1.predict_proba(x_test)[:,1] > 0.90).astype("int")

test_accuracy_1 = accuracy_score(y_test, test_preds1)

print(f"Valid accuracy score: {round(accuracy1, 4)}")
print()
print(f"Test accuracy score: {round(test_accuracy_1, 4)}")

Valid accuracy score: 0.7465

Test accuracy score: 0.6827


En este caso el desempeño en el conjunto de prueba bajó a un 68%

## Realizar prueba de cordura a los modelos

In [ ]:
np.random.seed(42)

sanity_check_1_preds = [0] * len(y_test)
sanity_check_2_preds = [1] * len(y_test)
sanity_check_3_preds = np.random.choice([0, 1], size=len(y_test))

print(f"Test accuracy score RandomForestClassifier: {round(test_accuracy_, 4)}")
print()
print(f"Test accuracy score LogisticRegression: {round(test_accuracy_1, 4)}")
print()
print(f"All 0 preds accuracy score: {round(accuracy_score(y_test, sanity_check_1_preds), 4)}")
print()
print(f"All 1 preds accuracy score: {round(accuracy_score(y_test, sanity_check_2_preds), 4)}")
print()
print(f"Random preds accuracy score: {round(accuracy_score(y_test, sanity_check_3_preds), 4)}")

Test accuracy score RandomForestClassifier: 0.7434

Test accuracy score LogisticRegression: 0.6827

All 0 preds accuracy score: 0.675

All 1 preds accuracy score: 0.325

Random preds accuracy score: 0.4386


In [ ]:
#Revisamos el balance de target
df.is_ultra.mean()

0.30647168637212197

El 30% de los datos son 1, esto significa un desequilibrio en las clases. Cuando existe un desequilibrio de clases el Sanity Check no es la métrica más adecuada para evaluar el rendimiento del modelo.

## Conclusión general

1.- En este estudio:
 - Examinamos los archivos con la siguiente dirección: "datasets/users_behavior.csv".
 - Segmentamos los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.
 - Investigamos la calidad de los modelos RandomForestClassifier y LogisticRegression
 - Comprobamos la calidad de los modelos usando el conjunto de prueba.
 - Finalmente realizamos una prueba de cordura a los modelos.

2.- Al ajustar los hiperparámetros de ambos modelos obtuvimos los siguientes resultados:
 - **RandomForestClassifier**
  - Accuracy score: 0.8243
  - Con los hiperparámetros: n_estimators=25, max_depth=9


 - **LogisticRegression**
  - Accuracy score: 0.7465
  - En este caso solo se modificó el hiperparámetro random_state. Al cambiar los otros solo bajaba la calidad del modelo.
  
3.- Al comprobar la calidad de los modelos usando el conjunto de prueba obtuvimos los siguientes resultados:
   - Para **RandomForestClassifier** la precisión del modelo bajó de un 82% a un 74%.
   - Para **LogisticRegression** la precisión del modelo bajó de un 74% a un 68%.
   - Esto probablemente se debe a que los modelos están sobreajustados.

4.- Además realizamos una prueba de cordura, pero encontramos un desequilibrio en las clases por lo que esta métrica no es adecuada para evaluar el rendimiento de este modelo.


<div class="alert alert-block alert-success">
    
# Comentarios generales
<b>Comentario del revisor</b> <a class="tocSkip"></a>
    
Buen trabajo, Jesús. Todo ha sido corregido y has aprobado un nuevo proyecto. ¡Felicitaciones!
</div>